In [ ]:
!spacy download pt_core_news_sm # Após instalação, reiniciar o ambiente do colab.

In [ ]:
import math
import os
import pickle
import re
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, precision_recall_fscore_support)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import spacy

spc = spacy.load("pt_core_news_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
default_dir = "/data/"
data_tcc_pos_neg = default_dir+'labeled_data/dataset_label_pos_neg.csv'
data_neg_emotions = default_dir+'labeled_data/plutchik_other_emotions.csv'


In [ ]:
data_base = pd.read_csv(data_tcc_pos_neg)
data['type'] = 'with_theme-' + data['sentiment']
data["cleaned_text"] = data[["cleaned_text"]].values.astype("U")
data = data.dropna()

In [ ]:
stemmer = nltk.stem.snowball.PortugueseStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc) if w[0]!='@')

In [ ]:
def create_splits(data):
    test_validation_size = int(0.30*data.shape[0])
    train, test = train_test_split(data, test_size=test_validation_size, random_state=42, stratify=data['type'])
    return train, test

  
train, test = create_splits(data)
print('Training samples:  ', train.shape[0])
print('Test samples:      ', test.shape[0])



In [ ]:
vectorizer = TfidfVectorizer(
    stop_words=nltk.corpus.stopwords.words('portuguese'), 
    analyzer=stemmed_words,
    min_df=0.0001, 
    max_features=100, 
    max_df=0.8,
    lowercase=True, 
    use_idf=True,
    ngram_range=(1, 2))

In [ ]:
X_train = vectorizer.fit_transform(train['cleaned_text'].values.astype('U'))
X_test = vectorizer.transform(test['cleaned_text'].values.astype('U'))

In [ ]:
# labels = {
#     'Positivo' : 0,
#     'Negativo' : 1,
# }

encoding = {
    'tristeza': 0,
    'medo': 1,
    'raiva': 2,
    'desprezo' : 3,
}


y_train = train['sentiment'].map(labels).values
y_test = test['sentiment'].map(labels).values

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))


In [ ]:
ax = sns.heatmap(confusion_matrix(y_test, y_pred), cmap='Greens_r', annot=True, fmt='d')
_ = ax.set(xlabel='Previsto', ylabel='Correto', title='Naive Bayes.')

In [ ]:
#Calculando a acurácia
acc = accuracy_score(y_pred, y_test)
precision, recall, fscore = precision_recall_fscore_support(y_pred, y_test, average="weighted")
print(f'Acurácia: {acc}')
print(f'\nPrecision: {precision} \n Recall: {recall}\n FScore: {fscore}')